# COVID-19 Dashboard

In [1]:
# Imports
from __future__ import print_function
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium

from ipywidgets import interact

In [2]:
# Covid Data Frames
death_df = pd.read_csv('../res/covid19_deaths_global.csv')
confirmed_df = pd.read_csv('../res/covid19_confirmed_global.csv')
recovered_df = pd.read_csv('../res/covid19_recovered_global.csv')
countries_df = pd.read_csv('../res/covid19_countries.csv')

In [3]:
# Columns Renaming
death_df = death_df.rename(columns = {'Province/State': 'State', 'Country/Region': 'Country'})
confirmed_df = confirmed_df.rename(columns = {'Province/State': 'State', 'Country/Region': 'Country'})
recovered_df = recovered_df.rename(columns = {'Province/State': 'State', 'Country/Region': 'Country'})

## Dataset of recoverded

In [4]:
recovered_df.head()

,State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Dataset of confirmed inceted

In [5]:
confirmed_df.head()

,State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


## Countries with the highest number of confirmed infected

In [6]:
def hightlighting(x:pd.DataFrame):
    c_confirmed = 'background-color: #9575CD'
    c_deaths = 'background-color: #EF5350'
    c_recovered = 'background-color: #4CAF50'
    temp_df = pd.DataFrame('', index=x.index, columns=x.columns)
    temp_df.iloc[:,4] = c_confirmed
    temp_df.iloc[:,5] = c_deaths
    temp_df.iloc[:,6] = c_recovered
    return temp_df


sorted_countries_df = countries_df.sort_values('Confirmed', ascending=False).head(5)
sorted_countries_df.style.apply(hightlighting, axis=None)

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Cases_28_Days,Deaths_28_Days
186,US,2023-03-10 13:21:02,40.000000,-100.000000,103804263,1123836,nan,nan,31506.793974,nan,nan,1.082649,840,USA,959794,9451.000000
80,India,2023-03-10 13:21:02,20.593684,78.962880,44690738,530779,nan,nan,3238.448985,nan,nan,1.187671,356,IND,5651,29.000000
63,France,2023-03-10 13:21:02,46.227600,2.213700,39866718,166176,nan,nan,61098.565402,nan,nan,0.416829,250,FRA,106042,618.000000
67,Germany,2023-03-10 13:21:02,51.165691,10.451526,38249060,168935,nan,nan,45997.289088,nan,nan,0.441671,276,DEU,355168,2275.000000
24,Brazil,2023-03-10 13:21:02,-14.235000,-51.925300,37085675,699310,nan,nan,17447.204607,nan,nan,1.885661,76,BRA,170852,1613.000000


In [7]:
fig = px.scatter(sorted_countries_df.head(10), x='Country_Region', y='Confirmed', size='Confirmed', color='Country_Region', hover_name='Country_Region', size_max=60)
fig.show()

## COVID-19 growth per day

In [8]:
def plot_cases_for_country(country):
    labels = ['Confirmed', 'Deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    df_list = [confirmed_df, death_df]
    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis=0)
        else:
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['Country']==country].iloc[:,5:]), axis=0)

        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
                                name=labels[i], connectgaps=True,
                                line = dict(color=colors[i], width=line_size[i]),
                                text=f"Totale {str(labels[i])}: {str(y_data[-1])}"
                    ))
    fig.show()

plot_cases_for_country('France')

interactive(children=(Text(value='France', description='country'), Output()), _dom_classes=('widget-interact',…

<function __main__.plot_cases_for_country(country)>

## Distribution of infected people on the map

In [9]:
wmap = folium.Map(location=[0.0, 0.0], tiles="cartodbpositron", zoom_start=2, max_zoom=6, min_zoom=2)

for i in range(len(confirmed_df)):
    lon, lat = confirmed_df.iloc[i]['Long'], confirmed_df.iloc[i]['Lat']
    radiay = int((np.log(confirmed_df.iloc[i, -1] + 1.00001)) + 0.2) * 25000
    if (not np.isnan(lat) and not np.isnan(lon)):
        folium.Circle(
            location=[lat, lon],
            radius=radiay,
            fill = True,
            fill_color = 'red',
            color = 'red',
            tooltip="<div style='margin: 0;'>"+
                "<h4 style='text-align: center; font-weight: bold'>" + confirmed_df.iloc[i]['Country'] + "</h4>" +
                "<hr style='margin: 10px; color: black;'>" +
                "<ul style='list-style-tape: square; align-item: left; padding-left: 20px; padding-right:20px'>" +
                    f"<li>Confirmed: {str(confirmed_df.iloc[i, -1])}</li>" +
                    f"<li>Deaths: {str(death_df.iloc[i, -1])}</li>" +
                "</ul></div>"
        ).add_to(wmap)

wmap